In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential, layers
import numpy as np

In [27]:
model = Sequential([
    layers.InputLayer(shape=(50,)),
    layers.Dense(100, 'relu'),
    layers.Dense(100, 'relu'),
    layers.Dense(100, 'softmax')
])

In [28]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 100)            │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 100)            │        10,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,300 (98.83 KB)

 Trainable params: 25,300 (98.83 KB)

 Non-trainable params: 0 (0.00 B)

Dropout

In [29]:
model = Sequential([
    layers.Dropout(0.999)
])

In [30]:
print(model(np.ones((1, 5))).numpy())
print(model(np.ones((1, 5)), training=True).numpy())

[[1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0.]]


Skip connection

In [31]:
input = keras.Input(shape=(50,))
out1 = layers.Dense(1000, 'relu')(input)
out2 = layers.Dense(1000, 'relu')(out1)
out3 = layers.Dense(1)(out2)
out4 = layers.Dense(1)(input)
output = out3 + out4
model = Model(inputs=input, outputs=output)
# можно добавить еще выход, например
model = Model(inputs=input, outputs=[output, out3])

Иерархические модели
Другие модели можно использовать как слой в другой

In [32]:
from tensorflow.keras.applications import Xception

model = Sequential([
    Xception(include_top=False, input_shape=(150, 150, 3)),
    layers.GlobalMaxPool2D(),
    layers.Dropout(0.5),
    layers.Dense(10)
])

In [33]:
print(len(model.layers))
print(len(model.layers[0].layers))

4
132


Один и тот же слой принадлежит нескольким моделям

In [34]:
# есть модель
model = Sequential([
    layers.InputLayer(shape=(400,)),
    layers.Dense(300),
    layers.Dense(200),
    layers.Dense(100)
])

In [35]:
# получаем выходное значение второго слоя

input_data = np.zeros((64, 400))
l1, l2, l3 = model.layers
layer2_output = l2(l1(input_data)).numpy()


In [ ]:
# или
submodel = Model(inputs=model.input(), outputs=model.layers[1].output)
layer2_output = submodel(input_data).numpy()

Save and load a model

In [25]:
model.save('models/model.keras')
model = keras.models.load_model('models/model.keras')

Read and change weights

In [37]:
# Обнуляем байасы полносвязного слоя с индексом 0
layer = model.layers[0]
W, b = layer.get_weights()
layer.set_weights((W, np.zeros_like(b)))

In [38]:
# При создании слоя не в составе модели веса не будут созданы. Параметр input_shape будет игнорирован. 
# Для создания весов нужно раз запустить слой, передав в него входные данные, либо вызвать метод
# .build(input_shape)

layer = layers.Dense(100)
print([x.shape for x in layer.get_weights()])

layer.build((20, )) # or layer(np.zeros((1, 20)))
print([x.shape for x in layer.get_weights()])

[]
[(20, 100), (100,)]


In [40]:
# create custom func activation
# f(x) = sin(x) + x

my_activation = lambda x: tf.math.sin(x) + x
l = layers.Dense(1000, activation=my_activation) 